# Initialization

In [1]:
import os
import torch, pyro, numpy as np 

from clipppy.globals import register_globals
from clipppy import load_config
from clipppy.patches import torch_numpy
from ruamel.yaml import YAML

import matplotlib.pyplot as plt
imkwargs = dict(extent=(-2.5, 2.5, -2.5, 2.5))

torch.cuda.set_device(0)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

register_globals(**globals())

ModuleNotFoundError: No module named 'clipppy.globals'

In [ ]:
import sys
sys.path.append('/home/eliasd/lensing/elias_utils')
from plotting import *
from channels import split_channels

### Load config

In [ ]:
SYSTEM_NAME = "ngc4414"

os.makedirs(SYSTEM_NAME, exist_ok=True)
BASEDIR = os.path.join(SYSTEM_NAME)
os.makedirs(BASEDIR, exist_ok=True)

In [ ]:
SOURCE_DIR = '../../mock_data/sources'
source_name = f'{SYSTEM_NAME}.npy'

config = load_config('config.yaml', base_dir=SOURCE_DIR,
                     source_name=source_name)

model = config.umodel
guide = config.guide

MASK = config.kwargs['defs']['mask']
NOISE = model.stochastic_specs['sigma_stat']
PARAMS = YAML().load(open('../paper-source/params.yaml'))[SYSTEM_NAME]

config.conditioning
config.guide

most importnat:
config.model (never use)
config.umodel: not conditioned on the data

----

# Generate mock data

#### Utilities

creates a mask for the observations

In [ ]:
from skimage.transform import downscale_local_mean
from scipy.ndimage import filters
from pyrofit.lensing.utils import Converter

def get_mask(img, sigma=1., th=1.):
    """
    Generate mask
    """
    smooth = filters.gaussian_filter(img.numpy(), sigma=sigma)
    return smooth > th

#### Set image as source

take the image and set the image as a srouce of the model
sources in the config.yaml will be filled

In [ ]:
imgsrc = config.kwargs['defs']['imagesrc']
model.sources = {'image': imgsrc}

#### True parameters for generating mock data

In [ ]:
PARAMS['truth']

#### Generate a 10nx x 10ny mock image and downsample it to create a mock image

want to have good resolation 4000x4000 of the mock obseration and then downsample it to 400x400

In [ ]:
UPSAMPLE = 10

# Save final grid
X, Y = model.X.clone(), model.Y.clone()

# Evaluate model over finer grid
#model._X, model._Y = get_meshgrid(
#    config.kwargs["defs"]["res"] / UPSAMPLE,
#    config.kwargs["defs"]["nx"] * UPSAMPLE,
#    config.kwargs["defs"]["ny"] * UPSAMPLE
#)
model.coerce_XY(*get_meshgrid(
    config.kwargs["defs"]["res"] / UPSAMPLE,
    config.kwargs["defs"]["nx"] * UPSAMPLE,
    config.kwargs["defs"]["ny"] * UPSAMPLE
))
model.mask = torch.full_like(model.X, True, dtype=bool, device="cuda:0")
model.stochastic_specs['sigma_stat'] *= UPSAMPLE
#model.coerce_XY()

# model.

In [ ]:
# Sample from the model
with pyro.poutine.trace() as tracer, pyro.condition(data=PARAMS['truth']):
    OBS = model()
OBSTRACE = tracer.trace

# Downsample observation
OBS = torch.tensor(downscale_local_mean(OBS, (UPSAMPLE, UPSAMPLE)))

# Downsample info from trace
data = dict(trace=OBSTRACE, obs=OBS, **{
    key: torch.tensor(downscale_local_mean(OBSTRACE.nodes[key]['value'], (UPSAMPLE, UPSAMPLE)))
    for key in ('mu', 'Xsrc', 'Ysrc')
})

# Never use it (never double information)
full_trace = 'False'
if not full_trace:
    # Throw recoverable information out
    trace_data = {}
    for k, v in obs_trace.nodes.items():
        if "_deterministic" in v["infer"] and v["infer"]["_deterministic"]:
            continue
        if k == config.umodel.observation_name:
            continue

        try:
            trace_data[k] = v["value"].detach().clone()
        except:
            trace_data[k] = v["value"]

    data["trace_data"] = trace_data
else:
    data["trace"] = OBSTRACE

In [ ]:
# Restore coarse grid
model.coerce_XY(X, Y)
model.mask = MASK
model.stochastic_specs['sigma_stat'] /= UPSAMPLE

#### Show mock observation

In [ ]:
plt.imshow(np.ma.array(data=OBS, mask=False), **imkwargs)
plt.colorbar()

#### Make a mask

In [ ]:
# Generate mask
model.mask = MASK = torch.tensor(get_mask(OBS, sigma=3., th=1.), dtype=bool)
print(f"{MASK.sum()} unmasked pixels ({MASK.sum().float() / MASK.numel() * 100}% of image)")
snr = (MASK * OBS).sum() / (NOISE * np.sqrt(MASK.sum().float()))
print(f"S/N: {snr}")

plt.imshow(MASK)

#### Save everything

In [ ]:
torch.save(data, os.path.join(BASEDIR, f"mock_sigma_stat=1.pt"))
np.save(os.path.join(BASEDIR, f"mask.npy"), MASK)